# 4. Создание сетевых файлов для Pajek

В этом ноутбуке производится трансформацция датафрейма с данными публикаций в TXT-файлы, содержащие вершины сети и связи между ними.

План работы:
- Создается список уникальных новых ID, каждому из них присуждается условный порядковый номер
- Итеративно анализируются строки датафрейма с публикациями, в итоговый файл записываются соответствующие указанным в строке публикации новым ID её авторов порядковые номера из списка выше
- Результаты сохраняются в TXT-формате

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
df = pd.read_parquet('/Users/vasilisavashenko/Downloads/df7.parquet')

In [ ]:
new_ids = []
for i in df.columns:
    if 'new_id' in i:
        new_ids += df[i].to_list()
new_ids = list(set(new_ids))
len(new_ids)

37791

In [ ]:
new_ids.remove('N/A')

In [ ]:
vertices = df['id'].to_list() + new_ids

vv = pd.DataFrame()
vv['number'] = list(range(1, len(vertices)+1))
vv['vertice'] = vertices
vv.head(10)

,number,vertice
0,1,15485609
1,2,15488335
2,3,15488853
3,4,15500786
4,5,15501422
5,6,15502590
6,7,15502840
7,8,15507187
8,9,15510295
9,10,15511500


In [ ]:
def get_index(search):
    return vv.loc[vv.isin([search]).any(axis=1)].index.tolist()[0]+1

In [ ]:
idd = [m for m in df.columns if 'new_id' in m]
arcs = pd.DataFrame(columns = ['v1', 'v2'])

for i in tqdm(df.index):
    for m in idd:
        if df[idd][m][i]!='N/A':
            arcs = arcs.append({'v1': i+1,
                                'v2': get_index(df[m][i])}, ignore_index = True)

In [ ]:
counter = [i for i in tqdm(vv['number'][75232:]) if i not in arcs['v2'].to_list()]
counter

In [ ]:
np.savetxt(r'/Users/vasilisavashenko/Downloads/arc+works4.txt', arcs.values, fmt='%s')
np.savetxt(r'/Users/vasilisavashenko/Downloads/vertices_fin3.txt', vv.values, fmt='%s')